# Process Raw Data

In this notebook, I'll process the raw metadata associated with photographs for each of the seven continents from the last nine year (2015-2024) downloaded from the [Macaulay library](https://search.macaulaylibrary.org/catalog?view=list). 

In [63]:
import os
import pandas as pd
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

Import the raw data and combine into a single dataset containing photographs from each continent.

In [64]:
data = "../data/raw/"
all_records_df = pd.concat([
    pd.read_csv(os.path.join(data, filename)).assign(
        continent=filename[:-4].replace('-', ' ').title()
    )
    for filename in os.listdir(data)
], ignore_index=True)

## Filter the raw data

We only need a few important columns containing species info, date, and photographer. We want to make sure we're getting *good* photos that are rated by *enough* people. Finally, we only want one photo for each photographer. It's unlikely that the same photographer will use multiple cameras. Professionals, maybe. We sure aren't.

In [65]:
# Keep only the relevant columns and rename them
columns_to_keep={
    "ML Catalog Number": "catalog_number",
    "Common Name": "common_name",
    "Scientific Name": "scientific_name",
    "Recordist": "photographer",
    "Date": "date",
    "Year": "year",
    "Average Community Rating": "rating",
    "Number of Ratings": "num_ratings",
    "continent": "location"
}

all_records_df = (
    all_records_df[[col for col in columns_to_keep.keys()]]
    .rename(columns=columns_to_keep)
)

In [80]:
# Filter to keep the best photos from each photographer
min_rating = 4
min_num_ratings = 20
top_records_df = (all_records_df
    .query(f'rating > {min_rating}')
    .query(f'num_ratings > {min_num_ratings}')
    .sort_values('rating', ascending=False)
    .drop_duplicates(subset='photographer') 
)

## Examine the filtered data

**How well is each continent represented?** How many species do we have represented?

In [84]:
alt.Chart(top_records_df).mark_bar().encode(
    x='location',
    y='count()',
    color='location'
).properties(
    title='Number of top-rated photos per continent',
    width=250,
    height=250
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

alt.Chart(...)

Unsurprisingly, North America is overrepresented and Africa and Antarctica are underrepresented. That's not necessarily a bad thing. Africa and Antarctica will probably over-index towards professionals and the very wealthy. I'm neither of those.

**What about species diversity?** It's not *that* important, but I'm curious how well this dataset represents bird diversity. It's much easier to get a 5-star photo of a duck than a warbler. We don't want to enrich for slow, fearless birds.

In [107]:
# Calculate species counts
species_counts = top_records_df['common_name'].value_counts().reset_index()
species_counts.columns = ['common_name', 'count']

# Create a selection for the count filter
count_filter = alt.binding_range(min=1, max=int(species_counts['count'].max()), step=1, name='Minimum Count: ')
count_selector = alt.param(name='CountThreshold', value=5, bind=count_filter)

# Create the chart with the filter
alt.Chart(species_counts).transform_filter(
    alt.datum.count >= count_selector
).mark_bar().encode(
    x=alt.X("common_name:N", title="Scientific Name", sort='-y'),
    y=alt.Y('count:Q', title='Number of Observations'),
    tooltip=[
        alt.Tooltip('common_name:N', title='Species'),
        alt.Tooltip('count:Q', title='Observations')
    ]
).properties(
    title="Representation of Bird Species in the Dataset",
    width=1000,
    height=500
).add_params(count_selector)

alt.Chart(...)

It's basically the usual suspects: owls, hummingbirds, shorebirds, and various other photogenic birds. I don't think it makes much sense to filter these out. They represent exactly the birds I'd love to have a great photo of!

**Finally, how well is each year represented?** I don't want to bias for only the newest equipment.

In [110]:
# Transform year into a string
top_records_df['year'] = top_records_df['year'].astype(str)

alt.Chart(top_records_df).mark_bar().encode(
    x='year',
    y='count()',
    color='location'
).properties(
    title='Number of top-rated photos per year',
    width=250,
    height=250
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

alt.Chart(...)

Earlier years are less represented, but that's not so unexpected. In total, we've got ~3,000 top rated photographs. The next step is to get the equipment metadata associated with each photo. This might be tricky, and will require scraping the web.

In [111]:
os.mkdir('../data/processed')
top_records_df.to_csv('../results/top_records.csv', index=False)